## Creating input-target pairs

In this section we implement a data loader that fetches the input-target pairs using sliding window approach

In [1]:
import importlib
import importlib.metadata
import tiktoken
print("tiktoken version: ", importlib.metadata.version("tiktoken"))

tiktoken version:  0.9.0


In [2]:
tokenizer = tiktoken.get_encoding('gpt2')

In [3]:
with open("the-verdict.txt", 'r', encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


we remove the first 50 tokens form the dataset for demonstration purposed as it results in slightly more interesting

In [4]:
enc_sample = enc_text[50:]

The context size determines the how many tokens are included in the input

In [5]:
context_size = 4 # length of the input
# the context size of 4 means that the model is trained to look at a sequence of 4 word
# to predict the next word in the sentence.
# the input x is the first 4 tokens [1, 2, 3, 4], and the target  is the next 4 token
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y: {y}")

x: [290, 4920, 2241, 287]
y: [4920, 2241, 287, 257]


In [6]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "------->", desired)

[290] -------> 4920
[290, 4920] -------> 2241
[290, 4920, 2241] -------> 287
[290, 4920, 2241, 287] -------> 257


Everything left of the arrow refers to the input and LLM would receive and the token ID on the right, LLM is supposed to predict

In [7]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "------->", tokenizer.decode([desired]))

 and ------->  established
 and established ------->  himself
 and established himself ------->  in
 and established himself in ------->  a


There is only one more task before we can turn the tokens into embeddings: implementing an efficient data loader that interates over the input datasets and returns the inputs and targets as pytorch tensors, which can be thought of  as multidimensional arrays

### Implementing Data Loader

step1: Tokenize the entire text

step2: Use a sliding window to chunk the book into overlapping sequences of max_length

setp3: Return the total number of rows in the dataset

step4: Return a single row from the dataset

In [8]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_lengths
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i+1:i+max_length +1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

The following code will use the GPTDatasetV1 to load the inputs in batches via a pytorch DataLoader

step1: Inintialize the tokenizer

step2: Create dataset

step3: drop_last=True drops the batch if it is shorter than the specified batch_size to prevent loss spikes during training

step4: the number of cpu processes to use for preprocessing

In [9]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

Lets test the dataloader with batch size of 1 for an LLM with context size of 4
This will develop an intuition of how the GPTDatasetV1 class and create_dataloader_v1 function work together:

In [10]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

### Convert dataloader into python iterator to fetch next entry via python's built-in next() function

In [11]:
import torch
print("Pytorch version:", torch.__version__)
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
                                  )
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

Pytorch version: 2.5.1+cu121
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


The first_batch variables contains the two tensors: the first tensor stores the input token IDs and the second tensor stores target token IDs

In [12]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [13]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False
                                  )
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])]
